# Table Orchestrator

In [0]:
unit = dbutils.widgets.get("unit")
tb_type = dbutils.widgets.get("tb_type")
print(unit)
print(tb_type)

In [0]:
from pyspark.sql.functions import *
from datetime import datetime
from delta.tables import DeltaTable
from pyspark.sql import Row

In [0]:
fileLookup = spark.read.table('inventory_project.metadata.filelookup')\
    .filter((col("schema") == "gold") & (col("description").startswith(unit)) & (col("description").contains(tb_type)))

In [0]:
def updateFileLookup(table_name, status):
    # Filter only the specific table row
    df = fileLookup.filter(col("table_name") == table_name) \
                   .withColumn("lastRunStatus", lit(status)) \
                   .withColumn("lastUpdatetime", current_timestamp())
    
    target_table = DeltaTable.forName(spark, "inventory_project.metadata.filelookup")

    # Perform merge (upsert)
    target_table.alias("t") \
        .merge(
            df.alias("s"),
            "t.table_name = s.table_name"
        ) \
        .whenMatchedUpdate(set={              
            "lastRunStatus": "s.lastRunStatus",
            "lastUpdatetime": "s.lastUpdatetime"
        }) \
        .whenNotMatchedInsertAll() \
        .execute()
   


In [0]:
def process_notebook(row):
    status = 'success'
    error = 'Nil'
    start_time = datetime.now()
    user = spark.sql("SELECT current_user()").collect()[0][0]
    name = row['table_name']
    path = row['source_path']
    try:
        print(f"Processing notebook {name}")
        dbutils.notebook.run(path,600,{"name":name})
    except Exception as e:
        status = 'fail'
        error = str(e)
        print(f"Error processing notebook {name}: {e}")
        raise e
    finally:
        end_time = datetime.now()
        print(f"Notebook {name} took {end_time-start_time} seconds to run")
        updateFileLookup(name,status)
        audit_rows.append(Row(table_name = name, 
                              start_time = start_time, 
                              end_time = end_time, 
                              duration = str((end_time-start_time).total_seconds()), 
                              status = status,
                              error_message = error,
                              created_by = user,
                              created_date = datetime.now()))


In [0]:
audit_rows = []
for row in fileLookup.collect():
    process_notebook(row)
if audit_rows:   # only if list is not empty
    audit_df = spark.createDataFrame(audit_rows)
    audit_df.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable("inventory_project.metadata.audit_log")
